In [1]:
# Import the required libraries and dependencies
import pandas as pd
import datetime as dt
from prophet import Prophet
from clean_data import clean_data


In [2]:
# Create path to CSV file
csv_path = "Metro_Interstate_Traffic_Volume.csv"
# Import the CSV into a pandas DataFrame
df = pd.read_csv(csv_path)
# Run imported DataFrame through clean_data function
clean_df = clean_data(df)

In [ ]:
sorted_df = clean_df.sort_values(by='date_time', ascending=True)
sorted_df.head(60)


In [4]:
def identify_gaps(df, output_file):
    # Define the expected range of hourly datetimes
    start_date = pd.to_datetime('2012-10-02 09:00:00')
    end_date = pd.to_datetime('2018-09-30 23:00:00')
    expected_dates = pd.date_range(start=start_date, end=end_date, freq='H')

    # Extract the actual datetimes from the DataFrame
    actual_dates = df.index

    # Identify missing datetimes
    missing_dates = expected_dates[~expected_dates.isin(actual_dates)]

    # Export missing datetimes to a CSV file
    missing_dates.to_frame(name='missing_datetime').to_csv(output_file, index=False)

# Assuming 'df' is your DataFrame
output_file = 'missing_datetimes.csv'
identify_gaps(clean_df, output_file)

C:\Users\Matt\AppData\Local\Temp\ipykernel_12868\2311151761.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  expected_dates = pd.date_range(start=start_date, end=end_date, freq='H')


In [ ]:
len(clean_df)

In [ ]:
holiday_rows = df['is_holiday']

In [ ]:
df['holiday'].unique()